---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [267]:
import pandas as pd
import numpy as np
import datetime
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(20)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [299]:
def date_sorter():
    
    pd.set_option('display.max_rows', 1000)
    
    # extract date information
    # normalize columns to month, day, year cols=['0', '1', '2']
    # merge data
    # convert date to datetime
    # sort
    # return as series
    
    cols=['Month', 'Day', 'Year']
    month_dict = dict({'Jan':1, 'January':1, 'Feb':2, 'February':2, 'Mar':3, 'March':3,
                      'Apr':4, 'April':4, 'May':5, 'Jun':6, 'June':6, 'Jul':7, 'July':7,
                      'Aug':8, 'August':8, 'Sep':9, 'September':9, 'Oct':10, 'October':10,
                      'Nov':11, 'November':11, 'Dec':12, 'December':12,
                       'Janaury':1, 'Decemeber':12})

    
    
    # Step_1
    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    # need to fis year 19xx or 20xx. if two digits,  convert months to text
    # make sure not to getxx/yy/zzz
    date1a = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    date1b = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    date1a.columns = cols
    date1b.columns = cols
    # prepend years with 19
    date1a['Year'] = '19' + date1a['Year'].astype(str)
    #print(date1)
    
    
    # Step_2
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    # need to remove some extraneous . Take first 3 letters of month
    date2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-. ]( *)(\d{1,2})[-, ] *(\d{4})\b')
    del date2[1]
    date2.columns = cols
    #print(date2)
    
    
    # Step_3
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # need to swap month and day
    date3 = df.str.extractall(r'(\d{1,2}) ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-., ] *(\d{4})\b')
    c = date3.columns
    date3 = date3[c[np.r_[1, 0, 2:len(c)]]]
    date3.columns = cols
    #print(date3)
    
    
    # Step_4a
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    # there aren't any of these in the dataset
    #date4.columns = cols
    #print(date4)
    
    
    # Step_4b
    # merge first four unique cases
    dates = pd.concat([date1a, date1b, date2, date3], ignore_index=False)
    #print(dates)
    
    
    # Step_5
    # Feb 2009; Sep 2009; Oct 2010
    # need to add day = 1, might need to exclude '20 Mar 2009'
    # problem with 228
    date5 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-., ] *(\d{4})')
    date5[2] = '1'
    c = date5.columns
    date5 = date5[c[np.r_[0, 2, 1]]]
    date5.columns = cols
    #print(date5)
    # merge with dates
    dates = pd.merge(dates, date5, left_index = True, right_index = True, how = 'outer')
    dates1 = dates.loc[:228]
    del dates1['Month_y']
    del dates1['Day_y']
    del dates1['Year_y']
    dates1.columns = cols
    dates2 = dates.loc[229:]
    del dates2['Month_x']
    del dates2['Day_x']
    del dates2['Year_x']
    dates2.columns = cols
    dates = pd.concat([dates1, dates2], ignore_index=False)
    #print(date)
    
    
    # Step_6
    # 6/2008; 12/2009
    # need to add day (1), might need to exclude 4/3/09
    # PROBLEM
    date6 = df.str.extractall(r'(\d{1,2})/(\d{4})')
    date6['Day'] = '1'
    c = date6.columns
    date6 = date6[c[np.r_[0, 2, 1]]]
    date6.columns = cols
    #print(date6)
    # merge with dates
    dates = pd.merge(dates, date6, left_index = True, right_index = True, how = 'outer')
    dates1 = dates.loc[:341]
    del dates1['Month_y']
    del dates1['Day_y']
    del dates1['Year_y']
    dates1.columns = cols
    dates2 = dates.loc[342:]
    del dates2['Month_x']
    del dates2['Day_x']
    del dates2['Year_x']
    dates2.columns = cols
    dates = pd.concat([dates1, dates2], ignore_index=False)
    #print(dates)
    
    
    # Step 7
    # 2009; 2010
    # need to add 1/1
    date7a = df.str.extractall(r'(19\d{2})')
    date7b = df.str.extractall(r'(20\d{2})')
    date7 = pd.concat([date7a, date7b], ignore_index=False)
    date7[1] = '1'
    date7[2] = '1'
    c = date7.columns
    date7 = date7[c[np.r_[2, 1, 0]]]
    date7.columns = cols
    #print(date7)
    
    # merge with dates
    dates = pd.merge(dates, date7, left_index = True, right_index = True, how = 'outer')
    #print(dates)
    dates.Month_x.fillna(dates.Month_y, inplace=True)
    dates.Day_x.fillna(dates.Day_y, inplace=True)
    dates.Year_x.fillna(dates.Year_y, inplace=True)
    del dates['Month_y']
    del dates['Day_y']
    del dates['Year_y']
    dates.columns = cols       
    #print(dates)
    
    
    # Step 8 Create Datetime
    # convert months to numbers
    dates['Month'].replace(month_dict, inplace=True)
    # add column for DateTime, convert columns to ints
    dates['DateTime'] = dates[['Year', 'Month', 'Day']].astype(int).apply(lambda s : datetime.datetime(*s),axis = 1)
    dates.sort_values(['DateTime'], ascending=[True], inplace=True)
    #print(dates)
    # get the index
    output = pd.Series(dates.index.get_level_values(0))
    #print(output)

    return output

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     342
61     237
62      23
63     204
64     258
65     315
66      27
67      93
68      17
69     488
70     303
71     283
72     395
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     369
82     493
83     318
84     239
85     148
86     105
87     336
88       6
89     200
90      81